In [ ]:
import os, sys
import numpy as np
import torch
import h5py
import time
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [ ]:
def chunks(folder_count, file, signal, chunk_size, slide_size=15, norm=None):
        data = []
        label = []
        f_label = []
        n_chunks = int((len(signal)-chunk_size)/slide_size) +  1
        for i in range(n_chunks):
            chunk = signal[int(i*slide_size) : int(chunk_size+(i*slide_size))]
            if norm =='inside_chunking':
                chunk = normalise_data(chunk)
            data.append(chunk)
            label.append([int(file)])
            f_label.append(folder_count)
        return data, label, f_label, n_chunks
    
def normalise_data(data, norm_type='min_max'):
    if norm_type=='z':
        m = np.mean(data) 
        sd = np.sqrt((sum(data-m)**2)/len(data))
        data = (data-m)/sd   
    elif norm_type=='min_max':
        data = (data - min(data))/(max(data)-min(data))
    return data

def load_data(fold, chunk_s, slide_s, dump=False, old=False, norm=False, split_percent=0.9):
        data_train, label_train = [], []
        data_test, label_test = [], []
        train_f, test_f = [], []
        chunks_count, full_label = [], []
        train_set, test_set = [], []
        data_val, label_val = [], []
        folder_count = 0
        files = sorted(os.listdir('data'))
        f_check = {0:[], 1:[], 'l0':[], 'l1':[]}
        
        for i in range(len(files)):
            file_label_ = int(files[i][-5])
            with h5py.File('data/'+files[i], 'r') as f:
                    data = np.array(f['reconstructed_calcium'])
            if file_label_ == 1:
                f_check[1].append(files[i])
                f_check['l1'].append(len(data))
            elif file_label_ == 0:
                f_check[0].append(files[i])
                f_check['l0'].append(len(data))

        temp0 = list(zip(f_check['l0'], f_check[0]))
        temp0.sort(reverse=True)
        f_check['l0'], f_check[0] = zip(*temp0)  
        temp0 = list(zip(f_check['l1'], f_check[1]))
        temp0.sort(reverse=True)
        f_check['l1'], f_check[1] = zip(*temp0)
        f1, f2, f3, f4, f5 = [], [], [], [], []
        
        for i in range(len(f_check['l1'])):
            if i%5 == 0: f1.append(f_check[1][i])
            elif i%5 == 1: f2.append(f_check[1][i])
            elif i%5 == 2: f3.append(f_check[1][i])
            elif i%5 == 3: f4.append(f_check[1][i])
            elif i%5 == 4: f5.append(f_check[1][i])
        for i in range(len(f_check[0])):
            if i%5 == 0: f1.append(f_check[0][i])
            elif i%5 == 1: f2.append(f_check[0][i])
            elif i%5 == 2: f3.append(f_check[0][i])
            elif i%5 == 3: f4.append(f_check[0][i])
            elif i%5 == 4: f5.append(f_check[0][i])
                
        if fold == '1': train_set = f2 + f3 + f4 + f5; test_set = f1;
        if fold == '2': train_set = f1 + f3 + f4 + f5; test_set = f2;
        if fold == '3': train_set = f1 + f2 + f4 + f5; test_set = f3;
        if fold == '4': train_set = f1 + f2 + f3 + f5; test_set = f4;
        if fold == '5': train_set = f1 + f2 + f3 + f4; test_set = f5;

        for file in train_set:
            with h5py.File('data/'+file, 'r') as f:
                data = np.array(f['reconstructed_calcium'])
            if norm=='before_chunking': data = normalise_data(data);
            data_, label, f_label, c_c = chunks(folder_count, file=file[-5], signal=data, 
                                                chunk_size=chunk_s, slide_size=slide_s, norm=norm)
            data_train.extend(data_); label_train.extend(label); train_f.extend(f_label)
            folder_count += 1
            
        for file in test_set:
            with h5py.File('data/'+file, 'r') as f:
                data = np.array(f['reconstructed_calcium'])
            if norm=='before_chunking': data = normalise_data(data);
            data_, label, f_label, c_c = chunks(folder_count, file=file[-5], signal=data, 
                                                chunk_size=chunk_s, slide_size=slide_s, norm=norm)
            data_test.extend(data_); chunks_count.append(c_c); label_test.extend(label)
            test_f.extend(f_label); full_label.append(int(file[-5]))
            folder_count += 1

        data_val = data_train[int(len(data_train)*split_percent):]
        data_train = data_train[:int(len(data_train)*split_percent)]
        label_val = label_train[int(len(label_train)*split_percent):]
        label_train = label_train[:int(len(label_train)*split_percent)]
        train_f = train_f[:int(len(train_f)*split_percent)]
        return data_train, label_train, data_test, label_test, data_val, label_val, train_f, test_f, train_set, test_set, chunks_count, full_label

In [ ]:
class Net(nn.Module):
    def __init__(self, feat, k, s, p, z):
        super(Net, self).__init__()
        self.conv = nn.Conv1d(1, feat, kernel_size=k,stride=s)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(p)
        self.feat = feat
        self.z = z
        self.linear = nn.Linear(feat*z,2)
    def forward(self, x):
        c_in = x.permute(0, 2, 1)
        out = self.conv(c_in)
        out = self.relu(out)
        c_out = self.pool(out)
        c_out = c_out.reshape(c_out.shape[0], self.feat*self.z)
        c_out = self.linear(c_out)
        return c_out

def train():   
    file_acc = {}
    correct, loss_ = 0, 0
    model.train()
    outs, labs = [], []
    for i, data in enumerate(trainloader, 0):
        inputs, labels, f = data
        optimizer.zero_grad()
        if cuda_enabled==True:
            inputs, labels = inputs.cuda(), labels.cuda().squeeze()
        else:
            inputs, labels = inputs, labels.squeeze()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        loss_ +=  loss.item()
        softmax_output = torch.exp(outputs)
        top_p, top_class = softmax_output.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        comp = equals.squeeze().cpu().detach().numpy()
        correct += torch.mean(equals.type(torch.FloatTensor)).item()
        for j in range(len(f)):
            file = f[j].item()
            if file not in file_acc:
                file_acc[file] = [0, 0]
            if comp[j] == True:
                file_acc[file][0]+=1
                file_acc[file][1]+=1
            else:
                file_acc[file][1]+=1
    for i in file_acc.keys():
        file_acc[i] = [train_set[i], file_acc[i][0]/file_acc[i][1]*100]
    return loss_/len(trainloader), correct/len(trainloader)*100, file_acc

def eval_(loader, mode='test'):
    file_acc = {}
    correct, loss_ = 0, 0
    model.eval()
    outs, labs = [], []
    for i, data in enumerate(loader, 0):
        if mode=='test':
            inputs, labels, f = data
        else:
            inputs, labels = data
        if cuda_enabled==True:
            inputs, labels = inputs.cuda(), labels.cuda().squeeze()
        else:
            inputs, labels = inputs, labels.squeeze()
        
        outputs = model(inputs.float())
        loss = criterion(outputs, labels)        
        loss_ +=  loss.item()
        softmax_output = torch.exp(outputs)
        top_p, top_class = softmax_output.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        correct += torch.mean(equals.type(torch.FloatTensor)).item()
        comp = equals.squeeze().cpu().detach().numpy()
        if mode=='test':
            for j in range(len(f)):
                file = f[j].item()
                if file not in file_acc:
                    file_acc[file] = [0, 0]
                if comp[j] == True:
                    file_acc[file][0]+=1
                    file_acc[file][1]+=1
                else:
                    file_acc[file][1]+=1
    if mode=='test':
        for i in file_acc.keys():
            file_acc[i] = [test_set[i-len(train_set)], file_acc[i][0]/file_acc[i][1]*100]
        return loss_/len(loader), correct/len(loader)*100, file_acc
    else:
        return loss_/len(loader), correct/len(loader)*100

In [ ]:
cuda_enabled = True
#train with hyperparameter tuning on validation set
for ss in [30]:
    for feat in [12]:
        for k in [10]:
            for s in [10]:
                for p in [2]:
                    avg = []
                    for fold in ['1', '2', '3', '4', '5']:
                        start = time.time()
                        ep = 0
                        data_train, label_train, data_test, label_test, data_val, label_val, train_f, test_f, train_set, test_set, chunks_count, full_label = load_data(chunk_s=300, slide_s=ss, fold=fold, norm='inside_chunking')
                        z = int(((len(data_train[0])-k)/s+1)//p)
                        if cuda_enabled==True:
                            model = Net(feat, k, s, p, z).cuda()
                        else:
                            model = Net(feat, k, s, p, z)
                        criterion = nn.CrossEntropyLoss()
                        optimizer = optim.Adam(model.parameters(), lr=0.01)
                        train_dataset = torch.utils.data.TensorDataset(torch.tensor(np.array(data_train)), torch.tensor(np.array(label_train)),torch.tensor(np.array(train_f)))
                        trainloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=64)
                        val_dataset = torch.utils.data.TensorDataset(torch.tensor(np.array(data_val)), torch.tensor(np.array(label_val)))
                        valloader = torch.utils.data.DataLoader(val_dataset, shuffle=True, batch_size=64)
                        
                        max_train_acc, max_val_acc = 0, 0
                        for epoch in range(100):
                            
                            loss, tr_accuracy, file_accuracy = train()
                            loss, accuracy = eval_(valloader, 'val')
                            
                            if accuracy>max_val_acc: 
                                max_train_acc = tr_accuracy
                                max_val_acc = accuracy 
                                ep = epoch
                                torch.save(model.state_dict(), 'cnnf8k5s1_f{}_w10s_s1s_.pth'.format(fold))
                            elif epoch-ep > 40: 
                                break
                        avg.append(max_val_acc)
                        print('fold {}, train acc: {}, val acc: {}, '.format(fold, round(max_train_acc, 2), round(max_val_acc, 2)), ss, feat, k, s, p, ep, time.time() - start)
                    print('avg: ', round(sum(avg)/5, 2))

In [ ]:
avg = []
ss = 30; k = 10; feat = 12; s = 10; p =2;
#model results on test set
for fold in ['1', '2', '3', '4', '5']:
    start = time.time()
    ep = 0
    data_train, label_train, data_test, label_test, data_val, label_val, train_f, test_f, train_set, test_set, chunks_count, full_label = load_data(chunk_s=300, slide_s=ss, fold=fold, norm='inside_chunking')
    z = int(((len(data_train[0])-k)/s+1)//p)
    if cuda_enabled==True:
        model = Net(feat, k, s, p, z).cuda()
    else:
        model = Net(feat, k, s, p, z)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    train_dataset = torch.utils.data.TensorDataset(torch.tensor(np.array(data_train)), torch.tensor(np.array(label_train)),torch.tensor(np.array(train_f)))
    trainloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=64)
    test_dataset = torch.utils.data.TensorDataset(torch.tensor(np.array(data_test)), torch.tensor(np.array(label_test)), torch.tensor(np.array(test_f)))
    testloader = torch.utils.data.DataLoader(test_dataset, shuffle=True, batch_size=64)
    max_train_acc, max_test_acc = 0, 0
    for epoch in range(100):
        loss, tr_accuracy, file_accuracy = train()
        if accuracy>max_train_acc: max_train_acc = accuracy
        if accuracy>max_test_acc: 
            max_train_acc = tr_accuracy
            max_test_acc = accuracy 
            ep = epoch
        elif epoch-ep > 40: 
            break
    avg.append(max_test_acc)
    print('fold {}, train acc: {}, test acc: {}, '.format(fold, round(max_train_acc, 2), round(max_test_acc, 2)), ss, feat, k, s, p, ep, time.time() - start)
print('avg: ', round(sum(avg)/5, 2))

In [ ]:
#get majority voting(file wise) values
fold = '1'
ss = 30
data_train, label_train, data_test, label_test, data_val, label_val, train_f, test_f, train_set, test_set, chunks_count, f_l = load_data(chunk_s=300, slide_s=ss, fold=fold, norm=True)
test_dataset = torch.utils.data.TensorDataset(torch.tensor(np.array(data_test)), torch.tensor(np.array(label_test)), torch.tensor(np.array(test_f)))
testloader = torch.utils.data.DataLoader(test_dataset, shuffle=False, batch_size=128)
from sklearn.metrics import confusion_matrix
feat, k, s, p = 12, 10, 10, 2
z = int(((len(data_train[0])-k)/s+1)//p)
model = Net(feat, k, s, p, z).cuda()
model.load_state_dict(torch.load('cnnf8k5s1_f{}_w10s_s1s_.pth'.format(fold)))
import matplotlib.pyplot as plt
import sklearn
#plot kernels 
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         if name == 'cnn.conv.weight':
#             data = param.data
# #             data = data.permute(2, 1, 0)
#             for i in range(12):
#                 plt.scatter(data[i][0].detach().cpu().numpy())
#                 plt.show()
#         #         plt.plot(param.data.detach().cpu().numpy())


criterion = nn.CrossEntropyLoss()
# model.load_state_dict('cnnf8k5s1_f1_w10s_s0.5s_.pth')
model.eval()
def eval_():
    file_acc_ = {}
    file_acc = {}
    model.eval()
    correct = 0
    loss_ = 0
    outs = []
    labs = []
    mv_count = [0]*11
    for i, data in enumerate(testloader, 0):
        
        inputs, labels, f = data
        inputs, labels = inputs.cuda(), labels.cuda().squeeze()
        
        outputs = model(inputs.float())
        loss = criterion(outputs, labels)        
        loss_ +=  loss.item()
        softmax_output = torch.exp(outputs)
        top_p, top_class = softmax_output.topk(1, dim=1)
        outs.extend(top_class.view(top_class.shape[0]).cpu().detach().numpy().tolist())
        labs.extend(labels.cpu().detach().numpy().tolist())
        equals = top_class == labels.view(*top_class.shape)
        correct += torch.mean(equals.type(torch.FloatTensor)).item()
        comp = equals.squeeze().cpu().detach().numpy()
        for j in range(len(f)):
            file = f[j].item()
            if file not in file_acc:
                file_acc[file] = []
            file_acc[file].append(top_class[j].cpu().detach().numpy())
            if file not in file_acc_:
                file_acc_[file] = [0, 0]
            if comp[j] == True:
                file_acc_[file][0]+=1
                file_acc_[file][1]+=1
            else:
                file_acc_[file][1]+=1
    for i in file_acc_.keys():
        file_acc_[i] = [test_set[i-len(train_set)], file_acc_[i][0]/file_acc_[i][1]*100]
    cd = [0]*11
    from sklearn.metrics import f1_score
    from sklearn.metrics import accuracy_score
    for j in range(1, 11):
        cl = []
        for i in file_acc.keys():
            l = len(file_acc[i])
            if l>9:
                seg = file_acc[i][:j*l//10]
            else:
                if j<=l:
                    seg = file_acc[i][:j*l//l]
                else:
                    seg = file_acc[i]
            seg_acc = sum(seg)/len(seg)
            if seg_acc > 0.5: #more ones than zeros
                cl.append(1)
            else:
                cl.append(0) #more zeros than ones
        print(round(accuracy_score(f_l, cl), 4))
    print('~~~~')
    for j in range(1, 11):
        cl = []
        for i in file_acc.keys():
            l = len(file_acc[i])
            if l>9:
                seg = file_acc[i][:j*l//10]
            else:
                if j<=l:
                    seg = file_acc[i][:j*l//l]
                else:
                    seg = file_acc[i]
            seg_acc = sum(seg)/len(seg)
            if seg_acc > 0.5: #more ones than zeros
                cl.append(1)
            else:
                cl.append(0) #more zeros than ones
        
        print(round(f1_score(f_l, cl), 4))
        print('majority:',sklearn.metrics.confusion_matrix(f_l, cl))
    print('chunkwise:',round(accuracy_score(labs, outs), 4))
    print('chunkwise:',sklearn.metrics.confusion_matrix(labs, outs))
    return loss_/len(testloader), correct/len(testloader)*100,  outs, labs, file_acc_
a = eval_()
a[-1]

In [ ]:
def get_random():
    r = np.random.randint(0, len(data_train))
    plt.plot(data_train[r])
    print(label_train[r])
get_random()

In [ ]:
fold = '5'
ss = 30
import sklearn.svm
from sklearn import metrics

In [ ]:
def get_svm_results_on_val(svm):
        fold = '1'
        data_train, label_train, data_test, label_test, data_val, label_val, train_f, test_f, train_set, test_set, chunks_count, f_l = load_data(chunk_s=300, slide_s=ss, fold=fold, norm=True)
        svm.fit(np.squeeze(data_train), np.array(label_train).ravel())
        y_pred = svm.predict(np.squeeze(data_val))
        print(fold, metrics.accuracy_score(np.array(label_val).ravel(), y_pred))
        
for gamma in ['scale', 'auto']:
    for degree in [2, 3, 4]:
        for coef in [0, 0.5, 1, 5]:
            for C in [0.1, 0.5, 1, 2]:
                for kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
                    print(gamma, degree, coef, C, kernel)
                    svm = sklearn.svm.SVC(kernel=kernel, gamma=gamma, coef0=coef, C=C, degree=degree)
                    get_svm_results_on_val(svm)

In [ ]:
def get_svm_results_on_val(svm, fold='1'):
        data_train, label_train, data_test, label_test, data_val, label_val, train_f, test_f, train_set, test_set, chunks_count, f_l = load_data(chunk_s=300, slide_s=ss, fold=fold, norm=True)
        svm.fit(np.squeeze(data_train), np.array(label_train).ravel())
        y_pred = svm.predict(np.squeeze(data_val))
        print(fold, metrics.accuracy_score(np.array(label_val).ravel(), y_pred))
for gamma in ['auto']:
    for degree in [3, 4]:
        for coef in [0, 0.5, 1, 5]:
            for C in [0.1, 0.5, 1, 2]:
                for kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
                    print(gamma, degree, coef, C, kernel)
                    svm = sklearn.svm.SVC(kernel=kernel, gamma=gamma, coef0=coef, C=C, degree=degree)
                    get_svm_results_on_val(svm)

In [ ]:
def get_svm_results_on_test(svm, fold='1'):
        data_train, label_train, data_test, label_test, data_val, label_val, train_f, test_f, train_set, test_set, chunks_count, f_l = load_data(chunk_s=300, slide_s=ss, fold=fold, norm=True)
        svm.fit(np.squeeze(data_train), np.array(label_train).ravel())
        y_pred = svm.predict(np.squeeze(data_test))
        print(fold, metrics.accuracy_score(np.array(label_test).ravel(), y_pred))
for fold in ['1', '2', '3', '4', '5']:
        get_svm_results_on_test(svm, fold)